In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
import os
import math
import warnings
import sys
warnings.simplefilter(action='ignore', category=FutureWarning)
excel_file_path = '/content/sample_data/6-7 to 8-9 Sep.xlsx'
xls = pd.ExcelFile(excel_file_path)
available_sheets = xls.sheet_names
Sheets = {
    "Azaad Digital": ["Azaad Anchors", "Azaad Anchors "],
    "Pukhtun Digital": ["Pukhtun Anchors", "Pakhtun Anchors"],
    "Kashmir Digital": ["Kashmir Anchors"],
    "Burak New": ["Burak Anchors", "Burak Anchors "]
}

formattedReporterData = pd.DataFrame(columns=["Name", "No of Vlogs", "Views", "Platform", "Channel", "Date"])
def dateLogicMaker3000(realSheet, designationData, formattedDataFinalized):
    dd = designationData.iloc[::-1].reset_index(drop=True)
    dd.columns = dd.iloc[1]
    dd = dd.drop([0,1]).reset_index(drop=True)
    file_name = os.path.splitext(os.path.basename(excel_file_path))[0]
    allDates = file_name.split("o", 1)[1].strip()
    allDates = " ".join(allDates.split())
    dates = allDates.split(" ")[0].strip()
    month = allDates.split(" ")[1].strip()
    year = allDates.split(" ")[2].strip() if len(allDates.split(" ")) > 2 else "2025"
    dd.columns = dd.columns.str.strip()
    name = dd["Name"]
    date = dates.split("-")
    loopStoper = int(date[2].strip()) if len(date) > 2 else int(date[1].strip())
    selectedDayArray = dataExtractFormat[monthStartDay]
    selectedDayArrayUsed = False
    normalEnteryDate = [3, 2, 2]
    i = 1
    j = 0
    k = 0
    while i <= loopStoper:
        fullDate = f"{i}/{month}/{year}"
        cols = dd.columns.tolist()
        keyColumnSelectors = ["Final Views", "Final Views ", "Final Views 48 Hrs", f"Final Views 48 Hrs({i} & {i+1})", "Total 48 hrs Views", f"48 hrs ({i}&{i+1})"]
        for col in cols:
            found = False
            if col in keyColumnSelectors:
                found = True
                print(f"\nFOUNDED: Column 'Final Views' as {col} in {realSheet} starting from {fullDate}")
                idx = cols.index(col) + 1
                dataPerDate = dd.iloc[:, :idx]
                dd = dd.iloc[:, idx:]
                dataPerDate = dataPerDate.fillna(0)
                if selectedDayArrayUsed == False:
                    if k == len(selectedDayArray):
                        selectedDayArrayUsed = True
                    else:
                        dataBreak = selectedDayArray[k]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + selectedDayArray[k]
                        k = k + 1
                if selectedDayArrayUsed == True:
                    if j == 0:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 1:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 2:
                        dataBreak = normalEnteryDate[j]
                        formattedDataFinalized = enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = 0
                    else:
                        print("ERROR: In using the normal sequence of date-wise entry")
                        break
                break
        if found == False:
            i = i + 1
    return found, formattedDataFinalized

def enterDataInMyFormat(realSheet, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized):
    keyPlatforms = {
        "Youtube": ["YT Views"],
        "Twitter (X)": ["Twitter Views"],
        "TikTok": ["TiktokViews", "Tiktok Views"],
        "Instagram": ["Instagram"],
        "Facebook":["FB Views"]
    }
    if "No of Vlogs" not in dataPerDate.columns:
        print(f"NOT FOUND: Column 'No of Vlogs' in {realSheet} for {fullDate} so created the column with default value: 1")
        dataPerDate["No of Vlogs"] = 1
    for platform, keyPlatformGroup in keyPlatforms.items():
        properDate = i
        keyPlatform = next((p for p in keyPlatformGroup if p in dataPerDate.columns), None)
        if keyPlatform is None:
            print(f"NOT FOUND / MISMATCHED: Key platform column in {realSheet} for {platform} at date: {properDate}/{month}/{year}")
        else:
            dataPerDate[keyPlatform] = np.ceil(dataPerDate[keyPlatform] / dataBreak).astype(int)
            dataPerDate["No of Vlogs"] = np.ceil(dataPerDate["No of Vlogs"] / dataBreak).astype(int)
            for l in range(dataBreak):
                newData = pd.DataFrame({
                    "Name": name,
                    "No of Vlogs": np.where(dataPerDate[keyPlatform] > 0, dataPerDate["No of Vlogs"], 0),
                    "Views": dataPerDate[keyPlatform],
                    "Platform": platform,
                    "Channel": realSheet,
                    "Date": f"{properDate}/{month}/{year}",
                    "Designation": "Reporter"
                })
                formattedDataFinalized = pd.concat([formattedDataFinalized, newData], ignore_index=True)
                properDate = properDate + 1
    return formattedDataFinalized

print("Enter the number according to the day, month started:")
print("Monday    : 1\nTuesday   : 2\nWednesday : 3\nThursday  : 4\nFriday    : 5\nSaturday  : 6\nSunday    : 7")
monthStartDay = input("Enter number: ")
dataExtractFormat = {
    "1": [2, 2],
    "2": [1, 2],
    "3": [2],
    "4": [1],
    "5": [],
    "6": [2, 2, 2],
    "7": [1, 2, 2]
}
for realSheet, sheetGroup in Sheets.items():
    existingSheet = next((s for s in sheetGroup if s in available_sheets), None)
    if existingSheet:
        df = pd.read_excel(xls, sheet_name=existingSheet)
        print(f"\n\nLOADED SHEET: {existingSheet} as {realSheet}")
        df = df.dropna(how="all").reset_index(drop=True)
        currentHeaders = df.columns.tolist()
        df.columns = df.iloc[0]
        df = df.drop([0]).reset_index(drop=True)
        df = pd.concat([pd.DataFrame([currentHeaders], columns=df.columns), df], ignore_index=True)
        rdInvert = df
        rdInvert = rdInvert.iloc[::-1].reset_index(drop=True)
        names = rdInvert["Name"]
        names = names.to_list()
        cut = None
        for row, name in enumerate(names):
            if "Reporters / Vloggers" in name:
                cut = row + 1
                break
        if cut is not None:
            reporterData = rdInvert.iloc[:cut].copy()
            otherData = rdInvert.iloc[cut:].copy()
        else:
            reporterData = None
            otherData = rdInvert
            print(f"NOT FOUND: Reporters data as 'Reporters / Vloggers' in the {realSheet}")
            break
        found, formattedReporterData = dateLogicMaker3000(realSheet, reporterData, formattedReporterData)
        if found == False:
            print(f"NOT FOUND: Column 'Final Views' in {realSheet} for Reporter")
            break
    else:
        print(f"NOT FOUND: Channel {realSheet}")
        break


from google.colab import files
print("GOOD JOB: Here is your file, go check your downloads")
formattedReporterData.to_excel("ReporterDataFormatted.xlsx", index=False)
files.download("ReporterDataFormatted.xlsx")

